In [18]:
import cv2
import matplotlib.pyplot as plt
import numpy as np 
import json

# Extract data infromation that can help us decide how to design the model and preprocess the data

## Verify true fps match

In [17]:
# Load videos verify fps match 
cap_train = cv2.VideoCapture('./data/images/train/train.mp4')
cap_test = cv2.VideoCapture('./data/images/test/test.mp4')
cap_val = cv2.VideoCapture('./data/images/val/val.mp4')

print(f"train fps: {cap_train.get(cv2.CAP_PROP_FPS)}")
print(f"test fps: {cap_test.get(cv2.CAP_PROP_FPS)}")
print(f"val fps: {cap_val.get(cv2.CAP_PROP_FPS)}")

cap_train.release()
cap_test.release()
cap_val.release()

train fps: 30.0
test fps: 30.0
val fps: 30.0


In [14]:
RES_H = 640
RES_W = 640 
FPS = 30 

## Check annotations, instances, occurences, count (ground truth statistics)

### Define helper functions

In [96]:
# Extract necessary data from json
def load_json_data(path):
    file = open('./data/annotations/instances_train.json')
    data = json.load(file)
    file.close()
    return data

In [97]:
ID_BOLT = 1
ID_NUT = 2

In [98]:
# 1 - bolt, 2 - nut 
train_data = load_json_data('./data/annotations/instances_train.json')
train_annotations = train_data['annotations']

In [128]:
# Frames with bolt instance (repetition allowed)
frames_with_bolt = [instance['image_id'] for instance in train_annotations if instance['category_id'] == ID_BOLT]
# Frames with nut instance (repetition allowed)
frames_with_nut = [instance['image_id'] for instance in train_annotations if instance['category_id'] == ID_NUT]
# Frames with nut both (repetition allowed)
frames_with_both = np.intersect1d(frames_with_bolt, frames_with_nut, assume_unique=False)


frames_with_bolt, frames_with_bolt_count = np.unique(frames_with_bolt, return_counts=True)
frames_with_nut, frames_with_nut_count = np.unique(frames_with_nut, return_counts=True)

### Percentage of videos with Bolts

In [126]:
# True frame - frame containint the object of interest
print(f'Percentage of frames with bolts: {round(len(frames_with_bolt)/72)}%')
print("Bolts per true frame: {:.2f} \n".format(np.mean(frames_with_bolt_count)))

print(f'Percentage of frames with nuts: {round(len(frames_with_nut)/72)}%')
print("Nuts per true frame: {:.2f} \n".format(np.mean(frames_with_nut_count)))

print(f'Percentage of frames with both: {round(len(frames_with_both)/72)}%')

Percentage of frames with bolts: 92%
Bolts per true frame: 2.23 

Percentage of frames with nuts: 50%
Nuts per true frame: 1.00 

Percentage of frames with both: 47%
